# Capstone 2 - Lending Club Loan Defaults Prediction¶

# Modeling


Modeling is the fifth step in the Data Science Method.  The following will be performed in this step:

1. Fit Models with Training Data Set
2. Review Model Outcomes — Iterate over additional models as needed.
3. Identify the Final Model

In [48]:
#load python packages
import os
import pandas as pd
import pandas.api.types as ptypes
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings # For handling error messages.
#warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

In [49]:
# Read data output created in the EDA step
df = pd.read_csv("../EDA/data/Cap2_step3_output.csv", dtype={'verification_status_joint':object,'hardship_type':object, 'hardship_reason':object, 'hardship_status':object, 'hardship_loan_status':object, 'settlement_status':object}, index_col="Unnamed: 0")
df.head(5)

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,initial_list_status,application_type,verification_status_joint,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_loan_status,debt_settlement_flag,settlement_status
0,2500,13.56,55000.0,18.24,0.0,1.0,0.0,45.0,1.0,4341,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
1,30000,18.94,90000.0,26.52,0.0,0.0,71.0,75.0,1.0,12315,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
2,5000,17.97,59280.0,10.51,0.0,0.0,0.0,0.0,0.0,4599,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
3,4000,18.94,92000.0,16.74,0.0,0.0,0.0,0.0,0.0,5468,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN
4,30000,16.14,57250.0,26.35,0.0,0.0,0.0,0.0,0.0,829,...,w,Individual,NaN,N,NaN,NaN,NaN,NaN,N,NaN


In [50]:
df.shape

(2150627, 87)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2150627 entries, 0 to 2260667
Data columns (total 87 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   loan_amnt                                   int64  
 1   int_rate                                    float64
 2   annual_inc                                  float64
 3   dti                                         float64
 4   delinq_2yrs                                 float64
 5   inq_last_6mths                              float64
 6   mths_since_last_delinq                      float64
 7   mths_since_last_record                      float64
 8   pub_rec                                     float64
 9   revol_bal                                   int64  
 10  revol_util                                  float64
 11  out_prncp                                   float64
 12  total_pymnt                                 float64
 13  total_rec_int              

In [52]:
df.describe()

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,deferral_term,hardship_amount,hardship_dpd,orig_projected_additional_accrued_interest,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,...,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06,2.150627e+06
mean,1.455534e+04,1.315755e+01,6.911525e+04,1.910219e+01,3.019622e-01,5.717709e-01,1.680994e+01,1.166151e+01,1.972904e-01,1.522785e+04,...,3.856550e-03,6.066408e-01,1.407915e-02,7.129102e-01,6.392043e-02,1.664888e+00,8.860760e-01,7.394659e+01,7.149019e-01,1.966361e-01
std,8.859252e+03,4.824323e+00,3.159676e+04,1.439575e+01,8.613410e-01,8.803719e-01,2.308542e+01,2.867152e+01,5.606472e-01,1.705110e+04,...,9.259225e-02,5.605731e+00,2.050347e-01,1.360392e+01,1.144538e+00,3.550980e+01,1.851189e+01,7.449337e+02,5.868766e+00,1.883835e+00
min,5.000000e+02,5.310000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,9.670000e+00,4.500000e+04,1.219000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.810000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.200000e+04,1.269000e+01,6.400000e+04,1.815000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.096600e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.000000e+04,1.599000e+01,8.800000e+04,2.476000e+01,0.000000e+00,1.000000e+00,3.000000e+01,0.000000e+00,0.000000e+00,1.927500e+04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.000000e+04,3.099000e+01,1.635000e+05,9.990000e+02,5.800000e+01,3.300000e+01,2.260000e+02,1.290000e+02,8.600000e+01,9.597540e+05,...,2.300000e+01,1.850000e+02,3.000000e+00,9.439400e+02,3.700000e+01,2.680890e+03,1.407860e+03,3.360100e+04,5.213500e+02,1.810000e+02


In [53]:
df['zip_code'].value_counts()

112xx    22865
945xx    21787
750xx    21749
606xx    20025
300xx    19156
         ...  
399xx        1
513xx        1
649xx        1
885xx        1
889xx        1
Name: zip_code, Length: 955, dtype: int64

## 'loan_status' is the response variable.
## Plan to predict fully paid and charged off status
## Features of interest are: 
'tot_cur_bal', 'annual_inc_joint', 'annual_inc', 'mths_since_last_record', 'mths_since_last_major_derog', 'collections_12_mths_ex_med', 'pub_rec_bankruptcies', 'tax_liens', '
## Features to not consider are
'title','zip_code','addr_state'



In [54]:
df_sel = df.drop(['title', 'zip_code', 'addr_state'], axis=1)

''' 
# Check these values in df_sel

loan_amnt                                   
annual_inc   
revol_bal  
tot_cur_bal
total_bal_il
max_bal_bc                                  
avg_cur_bal  
tax_liens      
total_bal_ex_mort   
total_bc_limit    
total_il_high_credit_limit     
revol_bal_joint        
hardship_amount      
hardship_last_payment_amount      
settlement_amount                  
'''

In [55]:
df_sel['loan_amnt'].value_counts()

10000    183149
20000    123816
12000    118794
15000    118790
5000      83452
          ...  
725           1
550           1
950           1
850           1
750           1
Name: loan_amnt, Length: 1571, dtype: int64

In [56]:
df_sel['annual_inc'].value_counts()

60000.0     87189
50000.0     76355
65000.0     64903
70000.0     62078
80000.0     59833
            ...  
44271.0         1
44269.0         1
94657.0         1
94659.0         1
131070.0        1
Name: annual_inc, Length: 83915, dtype: int64

In [57]:
df_sel['revol_bal'].value_counts()

0         11980
8           209
2           163
10          159
5235        158
          ...  
94025         1
96074         1
85837         1
110417        1
109518        1
Name: revol_bal, Length: 88143, dtype: int64

In [58]:
df_sel['tot_cur_bal'].value_counts()

0.0         69031
14186.0        41
20275.0        39
22831.0        38
23442.0        38
            ...  
309690.0        1
309689.0        1
309686.0        1
509276.0        1
381300.0        1
Name: tot_cur_bal, Length: 447363, dtype: int64

In [59]:
df_sel['total_bal_il'].value_counts()

0.0         986116
10000.0         98
15000.0         90
5000.0          83
5500.0          76
             ...  
339145.0         1
339113.0         1
339092.0         1
339074.0         1
84262.0          1
Name: total_bal_il, Length: 152044, dtype: int64

In [60]:
df_sel['max_bal_bc'].value_counts()

0.0         866592
8.0            544
3000.0         522
2000.0         495
4000.0         437
             ...  
27465.0          1
27460.0          1
27456.0          1
27453.0          1
131035.0         1
Name: max_bal_bc, Length: 30086, dtype: int64

In [61]:
df_sel['avg_cur_bal'].value_counts()

0.0         69048
2277.0        269
2442.0        267
2148.0        265
2025.0        265
            ...  
72094.0         1
72092.0         1
72087.0         1
72086.0         1
131042.0        1
Name: avg_cur_bal, Length: 80280, dtype: int64

In [62]:
df_sel['tax_liens'].value_counts()

0.0     2092608
1.0       39790
2.0       10776
3.0        3866
4.0        1673
5.0         851
6.0         443
7.0         204
8.0         135
9.0          81
10.0         62
11.0         32
12.0         19
13.0         12
14.0         11
15.0         11
17.0          7
18.0          6
22.0          5
21.0          5
16.0          5
19.0          2
20.0          2
61.0          2
24.0          2
46.0          2
45.0          2
37.0          1
53.0          1
52.0          1
48.0          1
39.0          1
34.0          1
85.0          1
29.0          1
28.0          1
27.0          1
25.0          1
23.0          1
63.0          1
Name: tax_liens, dtype: int64

In [63]:
df_sel['total_bal_ex_mort'].value_counts()

0.0         49942
24214.0        57
23068.0        57
20275.0        57
19095.0        56
            ...  
320651.0        1
194012.0        1
194009.0        1
320659.0        1
125310.0        1
Name: total_bal_ex_mort, Length: 194363, dtype: int64

In [64]:
df_sel['total_bc_limit'].value_counts()

0.0        73131
5000.0     16656
6000.0     14857
10000.0    14392
7000.0     14119
           ...  
38144.0        1
38143.0        1
38132.0        1
38130.0        1
29778.0        1
Name: total_bc_limit, Length: 18408, dtype: int64

In [65]:
df_sel['total_il_high_credit_limit'].value_counts()

0.0         324857
10000.0      13083
15000.0       9912
5000.0        8071
20000.0       7998
             ...  
211627.0         1
211619.0         1
211611.0         1
211608.0         1
194698.0         1
Name: total_il_high_credit_limit, Length: 179974, dtype: int64

In [66]:
df_sel['revol_bal_joint'].value_counts()

0.0         2045053
20804.0          10
21072.0           9
11771.0           9
10041.0           9
             ...   
29454.0           1
29455.0           1
29456.0           1
29458.0           1
130981.0          1
Name: revol_bal_joint, Length: 55621, dtype: int64

In [67]:
df_sel['hardship_amount'].value_counts()

0.00      2140534
94.59           5
48.56           5
53.05           5
69.90           5
           ...   
101.21          1
482.40          1
156.07          1
568.60          1
420.62          1
Name: hardship_amount, Length: 8553, dtype: int64

In [68]:
df_sel['hardship_last_payment_amount'].value_counts()

0.00      2140534
0.02           25
0.03           25
0.12           23
0.11           22
           ...   
482.10          1
575.35          1
157.82          1
73.23           1
373.61          1
Name: hardship_last_payment_amount, Length: 8388, dtype: int64

In [69]:
df_sel['settlement_amount'].value_counts()

0.00        2118442
5000.00          61
4000.00          49
3000.00          48
6000.00          47
             ...   
10857.25          1
10858.00          1
1048.80           1
1600.01           1
6226.00           1
Name: settlement_amount, Length: 21005, dtype: int64

In [70]:
df_sel['loan_status'].value_counts()

Fully Paid                                             994300
Current                                                867316
Charged Off                                            253510
Late (31-120 days)                                      20918
In Grace Period                                          8418
Late (16-30 days)                                        3533
Does not meet the credit policy. Status:Fully Paid       1886
Does not meet the credit policy. Status:Charged Off       715
Default                                                    31
Name: loan_status, dtype: int64

In [71]:
df_sel.shape

(2150627, 84)

In [72]:
# Dropping irrelevant loan_status

df_sel.reset_index()
df_sel = df_sel[~(df_sel['loan_status'] == 'In Grace Period')]
df_sel = df_sel[~(df_sel['loan_status'] == 'Current')]
df_sel = df_sel[~(df_sel['loan_status'] == 'Late (31-120 days)')]
df_sel = df_sel[~(df_sel['loan_status'] == 'Default')]
df_sel = df_sel[~(df_sel['loan_status'] == 'Late (16-30 days)')]
df_sel.shape

(1250411, 84)

In [73]:
df_sel['loan_status'].value_counts()

Fully Paid                                             994300
Charged Off                                            253510
Does not meet the credit policy. Status:Fully Paid       1886
Does not meet the credit policy. Status:Charged Off       715
Name: loan_status, dtype: int64

In [84]:
# Setting '%Fully Paid' to 0
# Setting '%Charges Off' to 1

df_sel['loan_status'] = df_sel['loan_status'].replace(['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'], '0')
df_sel['loan_status'] = df_sel['loan_status'].replace(['Charged Off', 'Does not meet the credit policy. Status:Charged Off'], '1')

In [85]:
df_sel['loan_status'].value_counts()

0    996186
1    254225
Name: loan_status, dtype: int64

In [87]:
df_sel['loan_status'] = pd.to_numeric(df_sel['loan_status'])

In [88]:
df_sel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1250411 entries, 100 to 2260664
Data columns (total 84 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   loan_amnt                                   1250411 non-null  int64  
 1   int_rate                                    1250411 non-null  float64
 2   annual_inc                                  1250411 non-null  float64
 3   dti                                         1250411 non-null  float64
 4   delinq_2yrs                                 1250411 non-null  float64
 5   inq_last_6mths                              1250411 non-null  float64
 6   mths_since_last_delinq                      1250411 non-null  float64
 7   mths_since_last_record                      1250411 non-null  float64
 8   pub_rec                                     1250411 non-null  float64
 9   revol_bal                                   1250411 non

In [89]:
# Create a list of object columns.  Exclude response variable 'loan_status'
cols_obj = []
for col in df_sel.columns:
    if df_sel[col].dtype == object:
        cols_obj.append(col)
#cols_obj.remove('loan_status')
cols_obj

['term',
 'grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'pymnt_plan',
 'initial_list_status',
 'application_type',
 'verification_status_joint',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_loan_status',
 'debt_settlement_flag',
 'settlement_status']

In [90]:
df_sel = pd.concat([df_sel.drop(cols_obj, axis=1), pd.get_dummies(df_sel[cols_obj])], axis=1)
df_sel.head(5)

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,hardship_loan_status_Current,hardship_loan_status_In Grace Period,hardship_loan_status_Issued,hardship_loan_status_Late (16-30 days),hardship_loan_status_Late (31-120 days),debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE
100,30000,22.35,100000.0,30.46,0.0,0.0,51.0,84.0,1.0,15603,...,0,0,0,0,0,1,0,0,0,0
152,40000,16.14,45000.0,50.53,0.0,0.0,0.0,0.0,0.0,34971,...,0,0,0,0,0,1,0,0,0,0
170,20000,7.56,100000.0,18.92,0.0,0.0,48.0,0.0,0.0,25416,...,0,0,0,0,0,1,0,0,0,0
186,4500,11.31,38500.0,4.64,0.0,0.0,25.0,0.0,0.0,4472,...,0,0,0,0,0,1,0,0,0,0
269,20000,17.97,57000.0,22.18,0.0,0.0,0.0,0.0,0.0,33356,...,0,0,0,0,0,1,0,0,0,0


In [91]:
# Using a sample size of 200,000 for modeling, due to Memory error when using all observations.
df_sample = df_sel.sample(200000)
df_sample.head()

,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,revol_bal,...,hardship_loan_status_Current,hardship_loan_status_In Grace Period,hardship_loan_status_Issued,hardship_loan_status_Late (16-30 days),hardship_loan_status_Late (31-120 days),debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE
725582,20000,12.99,40000.0,25.92,0.0,1.0,0.0,0.0,0.0,1513,...,0,0,0,0,0,1,0,0,0,0
1925228,10000,13.98,48000.0,33.38,0.0,1.0,0.0,0.0,0.0,33512,...,0,0,0,0,0,1,0,0,0,0
198908,25000,27.27,68750.0,23.70,0.0,1.0,0.0,0.0,0.0,25046,...,0,0,0,0,0,1,0,0,0,0
1227435,20000,13.99,72000.0,7.42,1.0,0.0,18.0,0.0,0.0,3575,...,0,0,0,0,0,1,0,0,0,0
1225709,30000,10.99,93400.0,29.74,0.0,0.0,0.0,0.0,0.0,35687,...,0,0,0,0,0,1,0,0,0,0


In [92]:
y = df_sample['loan_status']
X = df_sample.drop(['loan_status'], axis=1)

In [93]:
from sklearn.model_selection import train_test_split

y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [94]:
from sklearn import preprocessing as ppg

scaler = ppg.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_train_scaled

array([[ 0.06526986,  1.6523239 ,  0.43627091, ..., -0.10291255,
        -0.05915121, -0.10620326],
       [-1.50154051, -0.06782462, -0.92157476, ..., -0.10291255,
        -0.05915121, -0.10620326],
       [-0.80153968,  0.61812418, -0.92157476, ..., -0.10291255,
        -0.05915121, -0.10620326],
       ...,
       [-0.59600752, -1.30675368, -1.21254168, ..., -0.10291255,
        -0.05915121, -0.10620326],
       [-1.41813615, -0.27888579, -0.59827817, ..., -0.10291255,
        -0.05915121, -0.10620326],
       [-1.07260383, -0.54060164, -1.18021202, ..., -0.10291255,
        -0.05915121, -0.10620326]])

In [95]:
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-4.76858439e-01,  1.16688321e+00, -1.24487134e+00, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01],
       [-1.38239143e+00, -1.20589911e-01, -5.98278169e-01, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01],
       [-1.33473180e+00, -6.78246189e-02, -7.59926462e-01, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01],
       ...,
       [-2.62127971e-04, -1.14423658e+00, -2.29978698e-01, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01],
       [-1.19175291e+00,  5.65358885e-01, -1.40651963e+00, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01],
       [-1.54920014e+00,  7.76420054e-01, -1.72981622e+00, ...,
        -1.02912553e-01, -5.91512134e-02, -1.06203264e-01]])

# Logistic Regression

In [96]:
# Using K fold cross validation and Grid search to get the most accurate prediction

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]

param_grid = {'C': Cs}
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, param_grid, cv=5)
logreg_cv.fit(X_train_scaled, y_train)
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))
# Print the accuracy from the testing data.
print("Accuracy score is {}".format(accuracy_score(logreg_cv.predict(X_test_scaled), y_test)))


Tuned Logistic Regression Parameters: {'C': 100}
Best score is 0.9947733333333334
0.99564


In [97]:
from sklearn.metrics import confusion_matrix

y_pred_log = logreg_cv.predict(X_test_scaled)
pl_log = confusion_matrix(y_test,y_pred_log)
pl_log

array([[39950,    36],
       [  182,  9832]], dtype=int64)

In [98]:
from sklearn.metrics import classification_report

cl_rep_log = classification_report(y_test,y_pred_log)
print(cl_rep_log)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39986
           1       1.00      0.98      0.99     10014

    accuracy                           1.00     50000
   macro avg       1.00      0.99      0.99     50000
weighted avg       1.00      1.00      1.00     50000



In [99]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

ac = accuracy_score(y_test, y_pred_log)
print('Test Data Set Logistic Regression: Accuracy=%.3f' % (ac))

f1 = f1_score(y_test, y_pred_log, average='weighted')
print('Test Data Set Logistic Regression: f1-score=%.3f' % (f1))

roc_auc = roc_auc_score(y_test, y_pred_log)
print('Test Data Set Logistic Regression: roc_auc_score=%.3f' % (roc_auc))

Test Data Set Logistic Regression: Accuracy=0.996
Test Data Set Logistic Regression: f1-score=0.996
Test Data Set Logistic Regression: roc_auc_score=0.990


# Random Forest

In [101]:
from sklearn.ensemble import RandomForestClassifier

#clf = RandomForestClassifier(n_estimators=30, random_state = 1,n_jobs=-1)
#model_res = clf.fit(X_train_scaled, y_train)
#y_pred_ranf = model_res.predict(X_test_scaled)

param_grid = {'n_estimators': [10,20,30,40,50]}
clf = RandomForestClassifier(random_state = 1,n_jobs=-1)
clf_cv = GridSearchCV(clf, param_grid, cv=5)
clf_cv.fit(X_train_scaled, y_train)
y_pred_ranf = clf_cv.predict(X_test_scaled)
print("Tuned Random Forest Parameters: {}".format(clf_cv.best_params_)) 
print("Best score is {}".format(clf_cv.best_score_))
print("Accuracy score is {}".format(accuracy_score(y_test, y_pred_ranf)))


0.98752


In [102]:
y_pred_ranf

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [103]:
pl_ranf = confusion_matrix(y_test,y_pred_ranf)
pl_ranf

array([[39971,    15],
       [  609,  9405]], dtype=int64)

In [104]:
cl_rep_ranf = classification_report(y_test,y_pred_ranf)
print(cl_rep_ranf)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     39986
           1       1.00      0.94      0.97     10014

    accuracy                           0.99     50000
   macro avg       0.99      0.97      0.98     50000
weighted avg       0.99      0.99      0.99     50000



In [105]:
ac = accuracy_score(y_test, y_pred_ranf)
print('Test Data Set Random Forest: Accuracy=%.3f' % (ac))

f1 = f1_score(y_test, y_pred_ranf, average='weighted')
print('Test Data Set Random Forest: f1-score=%.3f' % (f1))

roc_auc = roc_auc_score(y_test, y_pred_ranf)
print('Test Data Set Random Forest: roc_auc_score=%.3f' % (roc_auc))

Test Data Set Random Forest: Accuracy=0.988
Test Data Set Random Forest: f1-score=0.987
Test Data Set Random Forest: roc_auc_score=0.969


# Gradient Boosting

In [106]:
from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
#for learning_rate in learning_rates:
#    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
#    gb.fit(X_train_scaled, y_train)
#    print("Learning rate: ", learning_rate)
#    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_scaled, y_train)))
#    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test_scaled, y_test)))
#    print()
    
param_grid = {'learning_rate': learning_rates}
gb = GradientBoostingClassifier(n_estimators=30, max_features=2, max_depth = 2, random_state = 0)
gb_cv = GridSearchCV(gb, param_grid, cv=5)
gb_cv.fit(X_train_scaled, y_train)
y_pred_gdbst = gb_cv.predict(X_test_scaled)
print("Tuned Gradient Boosting Parameters: {}".format(gb_cv.best_params_)) 
print("Best score is {}".format(gb_cv.best_score_))
print("Accuracy score is {}".format(accuracy_score(y_test, y_pred_gdbst)))

0.88156


In [108]:
#gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.25, max_features=2, max_depth = 2, random_state = 0)
#gb.fit(X_train_scaled, y_train)
#y_pred_gdbst = gb.predict(X_test_scaled)
#print(accuracy_score(y_test, y_pred_gdbst))

In [107]:
pl_gbst = confusion_matrix(y_test,y_pred_gdbst)
pl_gbst

array([[38361,  1625],
       [ 4297,  5717]], dtype=int64)

In [109]:
cl_rep_gbst = classification_report(y_test,y_pred_gdbst)
print(cl_rep_gbst)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     39986
           1       0.78      0.57      0.66     10014

    accuracy                           0.88     50000
   macro avg       0.84      0.77      0.79     50000
weighted avg       0.88      0.88      0.87     50000



In [110]:
ac = accuracy_score(y_test, y_pred_gdbst)
print('Test Data Set Gradient Boosting: Accuracy=%.3f' % (ac))

f1 = f1_score(y_test, y_pred_gdbst, average='weighted')
print('Test Data Set Gradient Boosting: f1-score=%.3f' % (f1))

roc_auc = roc_auc_score(y_test, y_pred_gdbst)
print('Test Data Set Gradient Boosting: roc_auc_score=%.3f' % (roc_auc))

Test Data Set Gradient Boosting: Accuracy=0.882
Test Data Set Gradient Boosting: f1-score=0.874
Test Data Set Gradient Boosting: roc_auc_score=0.765


# Compare Models

Based on the accuracy score and classification report, Logistic regression using 5 fold cross validation and Grid Search performed the best.

| Model | Accuracy| f1_score|roc_auc_score|
| --- | --- | --- | --- |
| Logistic Regression | 0.996 | 0.996 |0.990|
| Random Forest | 0.988 | 0.987 |0.969|
| Gradient Boosting | 0.882 | 0.874 |0.765|

# Create and compare models using pycaret